In [1]:
%load_ext autoreload
%autoreload 2
from convnet import ConvDipNet
from timeDistributed import TimeDistributed
from torchinfo import summary
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch
import sys; sys.path.insert(0, '../')
from esinet.forward import create_forward_model, get_info
from esinet import Simulation
from copy import deepcopy
from CNN_LSTM.util import *
from dipoleDataset import DipoleDataset
import os
import mne

2025-11-27 09:29:40.619915: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-27 09:29:40.680820: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-27 09:29:41.614799: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [190]:
import pandas as pd
save_dir = "/mnt/data/convdip/training_data"
sim_info = pd.read_pickle(os.path.join(save_dir, 'simulation_info.pkl'))
pos = np.load(os.path.join(save_dir, "dipole_pos.npy"))

positions_per_trial = sim_info['positions']
del sim_info
print(positions_per_trial)

target = positions_per_trial.iloc[1]
match = (pos == target)
row_eq = match.all(axis=1)

pos[np.where(row_eq)[0]]
pos = torch.tensor(pos)

0    [[-36.25040817260742, -28.20974349975586, 9.22...
0    [[44.33734130859375, -49.42319107055664, 20.72...
0    [[54.335540771484375, -52.5910530090332, -17.9...
0    [[-29.0865421295166, -93.31341552734375, -14.0...
0    [[-34.94701385498047, -10.915384292602539, 57....
                           ...                        
0    [[-30.389795303344727, -58.96854782104492, -7....
0    [[-48.558677673339844, -28.22390365600586, 19....
0    [[9.353612899780273, -43.6870231628418, 31.143...
0    [[37.58522033691406, -33.92412185668945, 63.67...
0    [[-20.511646270751953, 29.38741111755371, 48.1...
Name: positions, Length: 100000, dtype: object


In [15]:
def max_euclidean_distance(points):
    """
    Find the maximum Euclidean distance between any two points.
    
    Args:
        points: numpy array of shape (n, 3) containing 3D coordinates
    
    Returns:
        max_distance: float, the maximum distance
        indices: tuple (i, j), indices of the two points with max distance
    """
    # Compute all pairwise squared distances using broadcasting
    # diff shape: (n, n, 3)
    diff = points[:, np.newaxis, :] - points[np.newaxis, :, :]
    
    # Compute squared distances, shape: (n, n)
    sq_distances = np.sum(diff**2, axis=2)
    
    # Find the maximum
    max_sq_dist = np.max(sq_distances)
    max_distance = np.sqrt(max_sq_dist)
    
    # Get indices of maximum distance
    i, j = np.unravel_index(np.argmax(sq_distances), sq_distances.shape)
    
    return max_distance, (i, j)

def cdist(x, y):
    '''
    Input: x is a Nxd Tensor
        y is a Mxd Tensor
    Output: dist is a NxM matrix where dist[i,j] is the norm
        between x[i,:] and y[j,:]
    i.e. dist[i,j] = ||x[i,:]-y[j,:]||
    '''
    differences = x.unsqueeze(1) - y.unsqueeze(0)
    distances = torch.sum(differences**2, -1).sqrt()
    return distances


In [4]:
max_d, _ = max_euclidean_distance(pos)
max_d

distances = cdist(pos, )

169.42452954118374

In [6]:
data_dir = "/mnt/data/convdip/training_data/"
eeg_data_dir = os.path.join(data_dir, "eeg_data")
interp_data_dir = os.path.join(data_dir, "interpolated_eeg_data_for_lstm")
source_data_dir = os.path.join(data_dir, "source_data")
info_path = os.path.join(data_dir, "info.fif")
dataset = DipoleDataset(eeg_data_dir, interp_data_dir, source_data_dir, info_path, get_whole_trial=False)
test_size = 0.15
val_size = 0.15

test_amount, val_amount = int(dataset.__len__() * test_size), int(dataset.__len__() * val_size)

# this function will automatically randomly split your dataset but you could also implement the split yourself
gen = torch.Generator()
gen.manual_seed(0)
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [
            (dataset.__len__() - (test_amount + val_amount)), 
            test_amount, 
            val_amount], 
            generator=gen)

B = 512  # batch size
train_dataloader = torch.utils.data.DataLoader(
            train_set,
            batch_size=B,
            shuffle=True,
)
val_dataloader = torch.utils.data.DataLoader(
            val_set,
            batch_size=B,
            shuffle=True,
)
test_dataloader = torch.utils.data.DataLoader(
            test_set,
            batch_size=B,
            shuffle=True,
)

In [ ]:
from WHD import WHD

whd = WHD(pos, positions_per_trial)

eps = 1e-6
alpha = 4

def WHD(prob_map, gt_points, pos):
    n_est_pts = prob_map.sum()
    d_matrix = cdist(pos, gt_points)
    p_replicated = prob_map.view(-1,1).repeat(1, gt_points.shape[0])
    term_1 = (1 / (n_est_pts + eps)) * torch.sum(prob_map*torch.min(d_matrix, 1)[0])
    d_div_p = torch.min((d_matrix + eps) / (p_replicated**alpha + eps / max_d), 0)[0]
    d_div_p = torch.clamp(d_div_p, 0, max_d)
    term_2 = torch.mean(d_div_p, 0)
    return term_1 + term_2


device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")
for j, data in enumerate(tqdm(train_dataloader)):
    idx, sample, target = data
    #sample, target = sample.to(device, dtype=torch.float), target.to(device, dtype=torch.float)
    target_locations = positions_per_trial.iloc[idx]

    for b in range(B):
        prob_map = target[b, :]
        prob_map = torch.relu(prob_map)

        whd_class = whd.WHD(prob_map, int(idx[b]))
        print(whd_class)
        break

    break

  0%|          | 0/137 [00:01<?, ?it/s]

tensor(0., dtype=torch.float64)
tensor(1., dtype=torch.float64)
tensor(29.2278, dtype=torch.float64)
tensor(56.9732, dtype=torch.float64)
38425
torch.Size([3, 3])
tensor(29.2278, dtype=torch.float64)


In [187]:
for j, data in enumerate(tqdm(train_dataloader)):
    idx, sample, target = data
    for b in range(B):
        output = target[b,:]
        specific_target = target[b,:]
        specific_target2 = target[b+2, :]
        loss2 = -torch.matmul(output, specific_target2.T) + torch.log(torch.sum(torch.exp(torch.matmul(output.T, target.T)))) 
        loss = -torch.matmul(output, specific_target.T) + torch.log(torch.sum(torch.exp(torch.matmul(output.T, target.T))))
        print(loss)
        print(loss2)
        break
    break

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/137 [00:01<?, ?it/s]

tensor(8.5265e-14, dtype=torch.float64)
tensor(214.0000, dtype=torch.float64)
